In [1]:
!mkdir protT5 # root directory for storing checkpoints, results and fasta files with sample proteins
!mkdir protT5/protT5_checkpoint
!mkdir protT5/sec_struct_checkpoint 
!mkdir protT5/output 

mkdir: cannot create directory ‘protT5’: File exists
mkdir: cannot create directory ‘protT5/protT5_checkpoint’: File exists
mkdir: cannot create directory ‘protT5/sec_struct_checkpoint’: File exists
mkdir: cannot create directory ‘protT5/output’: File exists


In [10]:
seq_path = "./protT5/proteins.fasta"
per_protein_path = "./protT5/output/per_protein_embeddings.h5" 

In [19]:
!nvidia-smi

Wed Mar  6 20:34:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                     Off| 00000000:00:1B.0 Off |                    0 |
|  0%   19C    P8               16W / 300W|      2MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from transformers import T5EncoderModel, T5Tokenizer
import torch
import h5py
import time
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("Using {}".format(device))

Using cuda:0


In [4]:
class ConvNet( torch.nn.Module ):
    def __init__( self ):
        super(ConvNet, self).__init__()
        # This is only called "elmo_feature_extractor" for historic reason
        # CNN weights are trained on ProtT5 embeddings
        self.elmo_feature_extractor = torch.nn.Sequential(
                        torch.nn.Conv2d( 1024, 32, kernel_size=(7,1), padding=(3,0) ), # 7x32
                        torch.nn.ReLU(),
                        torch.nn.Dropout( 0.25 ),
                        )
        n_final_in = 32
        self.dssp3_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 3, kernel_size=(7,1), padding=(3,0)) # 7
                        )
        
        self.dssp8_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 8, kernel_size=(7,1), padding=(3,0))
                        )
        self.diso_classifier = torch.nn.Sequential(
                        torch.nn.Conv2d( n_final_in, 2, kernel_size=(7,1), padding=(3,0))
                        )
        

    def forward( self, x):
        # IN: X = (B x L x F); OUT: (B x F x L, 1)
        x = x.permute(0,2,1).unsqueeze(dim=-1) 
        x         = self.elmo_feature_extractor(x) # OUT: (B x 32 x L x 1)
        d3_Yhat   = self.dssp3_classifier( x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 3)
        d8_Yhat   = self.dssp8_classifier( x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 8)
        diso_Yhat = self.diso_classifier(  x ).squeeze(dim=-1).permute(0,2,1) # OUT: (B x L x 2)
        return d3_Yhat, d8_Yhat, diso_Yhat

In [5]:
def get_T5_model():
    model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc")
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False)
    return model, tokenizer

In [6]:
def read_fasta( fasta_path, split_char="!", id_field=0):
    seqs = dict()
    with open( fasta_path, 'r' ) as fasta_f:
        for line in fasta_f:
            # get uniprot ID from header and create new entry
            if line.startswith('>'):
                uniprot_id = line.replace('>', '').strip().split(split_char)[id_field]
                # replace tokens that are mis-interpreted when loading h5
                uniprot_id = uniprot_id.replace("/","_").replace(".","_")
                seqs[ uniprot_id ] = ''
            else:
                # repl. all whie-space chars and join seqs spanning multiple lines, drop gaps and cast to upper-case
                seq= ''.join( line.split() ).upper().replace("-","")
                # repl. all non-standard AAs and map them to unknown/X
                seq = seq.replace('U','X').replace('Z','X').replace('O','X')
                seqs[ uniprot_id ] += seq 
    example_id=next(iter(seqs))
    print("Read {} sequences.".format(len(seqs)))
    print("Example:\n{}\n{}".format(example_id,seqs[example_id]))

    return seqs

In [15]:
def get_embeddings( model, tokenizer, seqs, per_protein,
                   max_residues=4000, max_seq_len=1000, max_batch=100 ):

    results = {
               "protein_embs" : dict()
              }

    # sort sequences according to length (reduces unnecessary padding --> speeds up embedding)
    seq_dict   = sorted( seqs.items(), key=lambda kv: len( seqs[kv[0]] ), reverse=True )
    start = time.time()
    batch = list()
    for seq_idx, (pdb_id, seq) in enumerate(seq_dict,1):
        seq = seq
        seq_len = len(seq)
        seq = ' '.join(list(seq))
        batch.append((pdb_id,seq,seq_len))

        # count residues in current batch and add the last sequence length to
        # avoid that batches with (n_res_batch > max_residues) get processed 
        n_res_batch = sum([ s_len for  _, _, s_len in batch ]) + seq_len 
        if len(batch) >= max_batch or n_res_batch>=max_residues or seq_idx==len(seq_dict) or seq_len>max_seq_len:
            pdb_ids, seqs, seq_lens = zip(*batch)
            batch = list()

            # add_special_tokens adds extra token at the end of each sequence
            token_encoding = tokenizer.batch_encode_plus(seqs, add_special_tokens=True, padding="longest")
            input_ids      = torch.tensor(token_encoding['input_ids']).to(device)
            attention_mask = torch.tensor(token_encoding['attention_mask']).to(device)
            
            try:
                with torch.no_grad():
               
                    embedding_repr = model(input_ids, attention_mask=attention_mask)
            except RuntimeError:
                print("RuntimeError during embedding for {} (L={})".format(pdb_id, seq_len))
                continue


            for batch_idx, identifier in enumerate(pdb_ids): # for each protein in the current mini-batch
                s_len = seq_lens[batch_idx]
                
                emb = embedding_repr.last_hidden_state[batch_idx,:s_len]
                if per_protein: 
                    protein_emb = emb.mean(dim=0)
                    results["protein_embs"][identifier] = protein_emb.detach().cpu().numpy().squeeze()
    return results

In [8]:
def save_embeddings(emb_dict,out_path):
    with h5py.File(str(out_path), "w") as hf:
        for sequence_id, embedding in emb_dict.items():
            # noinspection PyUnboundLocalVariable
            hf.create_dataset(sequence_id, data=embedding)
    return None

In [16]:
# Load the encoder part of ProtT5-XL-U50 in half-precision (recommended)
model, tokenizer = get_T5_model()
seqs = read_fasta( seq_path )

# Compute embeddings and/or secondary structure predictions
results = get_embeddings( model, tokenizer, seqs, True)
save_embeddings(results["protein_embs"], per_protein_path)


Read 2 sequences.
Example:
sp|P04637|P53_HUMAN Cellular tumor antigen p53 OS=Homo sapiens OX=9606 GN=TP53 PE=1 SV=4
MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD


In [17]:
!pip install snowflake-snowpark-python[pandas]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [18]:
import json
import logging
import os
import re
import sys
import time 
import requests
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col
from snowflake.snowpark.exceptions import *
import snowflake.connector
from snowflake.snowpark.types import VectorType, StructType, StructField
from snowflake.snowpark.functions import col, lit, vector_l2_distance

/usr/local/lib/python3.8/dist-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1.dev0+ga6eabc2b.d20230220), please install a version that adheres to: 'pyarrow; extra == "pandas"'
  warn_incompatible_dep(


In [20]:
def get_token():
    with open('/snowflake/session/token', 'r') as f:
        return f.read()

connection_params = {
    'host': os.environ['SNOWFLAKE_HOST'],
    'port': os.environ['SNOWFLAKE_PORT'],
    'protocol': 'https',
    'account': os.environ['SNOWFLAKE_ACCOUNT'],
    'authenticator': 'oauth',
    'token': get_token(),
    'role': 'SYSADMIN',
    'warehouse': 'COMPUTE_WH',
    'database': 'BIONEMO_DB',
    'schema': 'PUBLIC'
}

In [21]:
session = Session.builder.configs(connection_params).create()
token = get_token()

In [22]:
import sys
import numpy
import numpy as np
import h5py
numpy.set_printoptions(threshold=sys.maxsize)
datasets = []
keys =[]
#i=0
with h5py.File('./protT5/output/per_protein_embeddings.h5', 'r') as f: 
    for file_key in f.keys():
        #i=i+1
        group = f[file_key]
        #if i==101:
          #break;
        if isinstance(group, h5py._hl.dataset.Dataset):
            keys.append(np.array(file_key))
            
            datasets.append(np.array(group))
           # print(np.array(group))
            continue
        for group_key in group.keys():
            #print(group_key)
            group2 = group[group_key]
            #print(group2)
            if isinstance(group2, h5py._hl.dataset.Dataset):
                keys.append(np.array(file_key))
                datasets.append(np.array(group2))
                continue
            for group_key2 in group2.keys():
                group3 = group2[group_key2]
                #print(group3)
                if isinstance(group3, h5py._hl.dataset.Dataset):
                   keys.append(np.array(file_key))
                   datasets.append(np.array(group3))
                   continue

In [23]:
!pip install bionemo

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [24]:
API_KEY="<your_key>"
API_HOST="https://api.bionemo.ngc.nvidia.com/v1"

In [25]:
from bionemo.api import BionemoClient
api = BionemoClient(api_key=API_KEY, api_host=API_HOST)
models=api.list_models()
models

[{'name': 'openfold', 'category': 'protein-structure', 'methods': ['predict']},
 {'name': 'alphafold2',
  'category': 'protein-structure',
  'methods': ['predict']},
 {'name': 'esmfold',
  'category': 'protein-structure',
  'methods': ['predict-no-aln']},
 {'name': 'jackhmmer', 'category': 'msa-calculation', 'methods': ['align']},
 {'name': 'protgpt2', 'category': 'protein-sequence', 'methods': ['generate']},
 {'name': 'moflow',
  'category': 'molecule',
  'methods': ['embeddings', 'decoder', 'generate']},
 {'name': 'megamolbart',
  'category': 'molecule',
  'methods': ['embeddings', 'generate']},
 {'name': 'molmim',
  'category': 'molecule',
  'methods': ['embeddings', 'decoder', 'generate']},
 {'name': 'diffdock',
  'category': 'molecular-docking',
  'methods': ['generate']},
 {'name': 'esm2-650m',
  'category': 'protein-embedding',
  'methods': ['embeddings']},
 {'name': 'esm2-3b',
  'category': 'protein-embedding',
  'methods': ['embeddings']},
 {'name': 'esm2-15b',
  'category': '

In [ ]:
for i in range(0,3): #len(datasets)-1
    v=list(datasets[i])
    df=session.sql(f'''SELECT uniprotid, emb::VECTOR(FLOAT,1024), {v}::VECTOR(FLOAT,1024),
                        VECTOR_L2_DISTANCE(emb, {v}::VECTOR(FLOAT,1024)) as distance
                   from BIONEMO_DB.PUBLIC.PROTEINS
                   order by distance asc
                   limit 4''')
    df.show()
    proteins_df=df.select(col('UNIPROTID')).to_pandas()
    p=proteins_df.values.tolist()
    for i in range(1, len(p)):
        original_sequence = api.get_uniprot(p[i][0])
        original_result=api.openfold_sync(original_sequence)
        pdb_filename=f'''pdb/{p[i][0]}.pdb'''
        with open(pdb_filename, "w") as f:
            f.write(original_result)

-------------------------------------------------------------------------------------------------------------------------------------------------
|"UNIPROTID"  |"EMB::VECTOR(FLOAT,1024)"                           |"[0.01560472, 0.038481202, 0.020373046, 0.01996...  |"DISTANCE"             |
-------------------------------------------------------------------------------------------------------------------------------------------------
|P04637       |[0.01561999972909689, 0.03844999894499779, 0.02...  |[0.015604720450937748, 0.038481201976537704, 0....  |0.0006604380983691511  |
|Q9TTA1       |[0.014779999852180481, 0.04149999842047691, 0.0...  |[0.015604720450937748, 0.038481201976537704, 0....  |0.085698190101902      |
|P56424       |[0.012190000154078007, 0.03472999855875969, 0.0...  |[0.015604720450937748, 0.038481201976537704, 0....  |0.0992041789266408     |
|P61260       |[0.012190000154078007, 0.03472999855875969, 0.0...  |[0.015604720450937748, 0.038481201976537704, 0....  |0.0